# Exercise 3: MD5 Collisions and Password Cracking

## Exercise Objectives
1. Finding MD5 collisions (first 6 characters)
2. Recognizing hashing schemes (MD5-crypt, SHA256-crypt, Argon2)
3. Performing dictionary attacks
4. Handling hashes with salt suffix (pepper)

## Part 1: MD5 Collisions

### What is MD5?
**MD5 (Message Digest Algorithm 5)** is a cryptographic hash function that produces a 128-bit (16-byte) hash value, typically represented as a 32-character hexadecimal string.

Properties:
- **Deterministic**: Same input always produces same output
- **Fixed size**: Always 128 bits regardless of input size
- **One-way**: Computationally infeasible to reverse
- **Avalanche effect**: Small input change drastically changes output

### What is a Collision?
A collision occurs when two different input values produce the same hash output:

```
MD5("password1") = abc123...
MD5("password2") = abc123...  ← Collision!
```

**Why it matters**: Breaks hash uniqueness assumption, compromises integrity verification.

### Birthday Paradox
The probability of finding a collision grows much faster than intuition suggests:

$$P(\text{collision}) \approx 1 - e^{-\frac{n^2}{2m}}$$

where:
- $n$ = number of attempts (hashes computed)
- $m$ = number of possible hash values

### Partial Collision Attack
For this exercise, we target **first 6 hex characters** (24 bits):
- Search space: $m = 16^6 = 16,777,216$ possible values
- Expected attempts: $\sqrt{\pi m / 2} \approx 4,096$ for 50% probability
- With good luck: Could find in ~2,000-5,000 attempts

### Attack Strategy
```python
1. Generate random passwords: "pass0", "pass1", "pass2", ...
2. Compute MD5 for each: hash_dict[prefix] = password
3. Check if prefix already seen → COLLISION!
4. Continue until collision found
```

**Complexity**: O(√m) with O(n) memory

## Part 2: Password Hashing Schemes

### Why Hash Passwords?
Never store passwords in plaintext! Hashing provides:
- **One-way transformation**: Can't recover original password
- **Verification**: Compare hash(input) with stored hash
- **Breach protection**: Even if database leaked, passwords not immediately exposed

### Salt: Defense Against Rainbow Tables
**Salt** = Random value added to password before hashing:
```
hash = H(password || salt)
```

**Without salt**: Precomputed tables (rainbow tables) allow instant lookups
**With salt**: Each password needs unique computation, makes precomputation infeasible

### MD5-crypt ($1$)
Format: `$1$salt$hash`

**Structure breakdown**:
- `$1$` → Algorithm identifier (MD5-crypt)
- `salt` → 8-character random salt
- `hash` → 22-character base64-encoded hash

**Properties**:
- Uses 1,000 iterations of MD5
- Introduced in 1994 for Unix systems
- **Status**: Obsolete, too fast to compute
- **Vulnerability**: Modern GPUs can test millions of passwords/second

**Example**: `$1$abcd1234$e5fskK1p2kLxY.Z9wMFvg/`

### SHA256-crypt ($5$)
Format: `$5$rounds=N$salt$hash`

**Structure**:
- `$5$` → Algorithm identifier (SHA256-crypt)
- `rounds=N` → Number of iterations (optional, default 5,000)
- `salt` → Up to 16-character salt
- `hash` → 43-character base64-encoded hash

**Properties**:
- Uses SHA-256 hash function
- Configurable iteration count (1,000 to 999,999,999)
- Stronger than MD5-crypt but still crackable
- **Status**: Acceptable for low-value systems, not recommended for new deployments

**Example**: `$5$rounds=5000$saltsalt$kDV8h2KL7eDL.z2W0nNFKz1oS.xM7dQn`

### Argon2 ($argon2id$)
Format: `$argon2id$v=19$m=65536,t=3,p=4$salt$hash`

**Structure**:
- `$argon2id$` → Variant (argon2i, argon2d, or argon2id)
- `v=19` → Version number
- `m=65536` → Memory cost (KiB)
- `t=3` → Time cost (iterations)
- `p=4` → Parallelism degree (threads)
- `salt` → Base64-encoded salt
- `hash` → Base64-encoded hash

**Why Argon2 is Superior**:
1. **Memory-hard**: Requires large RAM, resists GPU/ASIC attacks
2. **Configurable**: Tune time/memory/parallelism to security needs
3. **Side-channel resistant**: argon2id variant defends against timing attacks
4. **Winner**: 2015 Password Hashing Competition

**Security parameters**:
- Minimum: m=15360 (15 MB), t=2, p=1
- Recommended: m=65536 (64 MB), t=3, p=4
- High security: m=262144 (256 MB), t=5, p=4

**Example**: `$argon2id$v=19$m=65536,t=3,p=4$c29tZXNhbHQ$RdescudvJCsgt3ub+b+dWRWJTmaaJObG`

### Comparison Table

| Scheme | Year | Iterations | GPU Attack | Status |
|--------|------|-----------|-----------|---------|
| MD5-crypt | 1994 | 1,000 | Easy | Obsolete |
| SHA256-crypt | 2008 | 5,000+ | Moderate | Acceptable |
| Argon2 | 2015 | Configurable | Resistant | Recommended |

## Part 3: Dictionary Attack

### What is a Dictionary Attack?
Systematic attempt to crack password hashes by testing common passwords from a list (dictionary).

**Attack flow**:
```
1. Load dictionary: ["password", "123456", "admin", ...]
2. For each candidate password:
   a. Extract salt from target hash
   b. Compute hash(candidate, salt)
   c. Compare with target hash
   d. If match → PASSWORD FOUND!
```

### Why Dictionary Attacks Work
**Human password patterns**:
- Use common words, names, dates
- Predictable substitutions (a→@, e→3)
- Short lengths (8-12 characters)
- Reuse across sites

**Statistics**:
- ~80% of passwords in leaked databases are in common dictionaries
- Top 10,000 passwords cover ~30% of all users
- "password", "123456", "qwerty" remain most popular

### Attack Complexity
For dictionary of size $D$:
- **Without pepper**: $O(D)$ attempts
- **With 1-char pepper [a-z]**: $O(26 \times D)$ attempts
- **With 2-char pepper**: $O(26^2 \times D) = O(676 \times D)$ attempts

### Pepper: Additional Secret Ingredient
**Pepper** = Secret value appended/prepended to password before hashing:
```
hash = H(password || pepper, salt)
```

**Key differences from salt**:
| Feature | Salt | Pepper |
|---------|------|--------|
| Storage | With hash (public) | Separate secure location (secret) |
| Unique per password | Yes | No (shared) |
| Purpose | Prevent rainbow tables | Slow dictionary attacks |

**In this exercise**:
- Pepper = single character from [a-z]
- Appended to password: `"alibaba" + "x"` → `"alibabax"`
- Attacker must test all 26 possibilities per dictionary word

### Attack Strategy with Pepper
```python
for password in dictionary:
    for pepper in 'abcdefghijklmnopqrstuvwxyz':
        candidate = password + pepper
        if verify(candidate, hash):
            return candidate
```

**Complexity**: 26× slower, but still feasible for small pepper space

### Optimization Techniques
1. **Parallel processing**: Distribute dictionary across multiple threads/cores
2. **GPU acceleration**: Use CUDA/OpenCL for massive parallelism
3. **Smart ordering**: Try most common passwords first
4. **Rule-based mutations**: Apply common transformations (capitalize, append numbers)

### Time Estimates (MD5-crypt, single CPU core)
- 10,000 word dictionary: ~1-2 seconds
- 100,000 words: ~10-20 seconds
- With 26× pepper multiplier: ~5-10 minutes
- Argon2 (high params): Hours to days

## Implementation Details

### Hash Format Parsing
Each scheme has distinct identifier at the beginning:

```python
def identify_hash_type(hash_string):
    if hash_string.startswith('$1$'):
        return 'MD5-crypt'
    elif hash_string.startswith('$5$'):
        return 'SHA256-crypt'
    elif hash_string.startswith('$argon2'):
        return 'Argon2'
    else:
        return 'Unknown'
```

### Extracting Salt from Hash
**MD5-crypt example**: `$1$abcd1234$e5fskK1p2kLxY.Z9wMFvg/`
```python
parts = hash_string.split('$')
# parts = ['', '1', 'abcd1234', 'e5fskK1p2kLxY.Z9wMFvg/']
algorithm = parts[1]  # '1'
salt = parts[2]       # 'abcd1234'
hash_value = parts[3] # 'e5fskK1p2kLxY.Z9wMFvg/'
```

**SHA256-crypt**: May include rounds parameter
```python
# Format: $5$rounds=5000$saltsalt$hash...
if 'rounds=' in parts[2]:
    rounds_str = parts[2]  # 'rounds=5000'
    salt = parts[3]        # 'saltsalt'
else:
    salt = parts[2]        # Default rounds
```

**Argon2**: More complex parameter extraction
```python
# $argon2id$v=19$m=65536,t=3,p=4$salt$hash
variant = parts[1]        # 'argon2id'
version = parts[2]        # 'v=19'
params = parts[3]         # 'm=65536,t=3,p=4'
salt = parts[4]           # base64-encoded
hash_value = parts[5]     # base64-encoded
```

### Manual Hash Computation (Passlib)
Instead of using `verify()`, manually compute hash for learning:

```python
from passlib.hash import md5_crypt, sha256_crypt

# Extract salt from target hash
salt = target_hash.split('$')[2]

# Compute hash with same salt
test_hash = md5_crypt.using(salt=salt).hash(password)

# Compare full hashes
if test_hash == target_hash:
    print(f"Password found: {password}")
```

### Why Manual Computation?
1. **Educational**: Understand how verification works internally
2. **Control**: See exact hash values being compared
3. **Debugging**: Easier to troubleshoot issues
4. **Transparency**: No "magic" happening behind the scenes

### Dictionary Loading
```python
def load_dictionary(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        # Read all lines, strip whitespace, filter empty
        passwords = [line.strip() for line in f if line.strip()]
    return passwords
```

### Progress Tracking (Optional)
For large dictionaries, show progress:
```python
from tqdm import tqdm  # Progress bar library

for password in tqdm(dictionary, desc="Testing passwords"):
    if test_password(password, target_hash):
        return password
```

### Error Handling
```python
try:
    result = crack_hash(hash_string, dictionary)
except ValueError as e:
    print(f"Invalid hash format: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
```

## Security Analysis

### Why These Hashes Are Weak

#### 1. Weak Password Selection
**Problem**: Passwords chosen from common dictionary
```
Examples: "password", "123456", "admin", "welcome"
```

**Why it's bad**:
- Appear in every attacker's dictionary
- Easily guessable through brute force
- No randomness or complexity

**Fix**: Use **password generators** with high entropy:
```
Good: "X7$mK9#pL2@qR5^wN3&vB8*"
Bad:  "password123"
```

#### 2. Obsolete Algorithms (MD5-crypt)
**Problem**: Only 1,000 MD5 iterations (1994 standard)

**Modern hardware performance** (single GPU):
- MD5: ~200 billion hashes/second
- MD5-crypt: ~20 million passwords/second
- Time to crack 8-char password [a-z]: ~20 minutes

**Fix**: Use Argon2 with high parameters (300× slower)

#### 3. Insufficient Iteration Counts
**Problem**: SHA256-crypt with default 5,000 rounds

**Attack speed**:
- 5,000 rounds: ~300,000 passwords/second (GPU)
- 100,000 rounds: ~15,000 passwords/second
- Argon2: ~100 passwords/second

**Fix**: Tune iteration counts to require ~0.5-1 second per hash

#### 4. Short Pepper Space
**Problem**: 26 possible values [a-z]

**Attack impact**:
- Only 26× multiplier on attack time
- For fast hashes (MD5-crypt): seconds → minutes
- Still easily brute-forced

**Fix**: Use **64+ character random pepper** from full ASCII range

### Defense Strategies

#### Level 1: User Education
- **Minimum 12 characters** (NIST recommendation)
- Mix uppercase, lowercase, numbers, symbols
- Avoid dictionary words, personal information
- Use password managers (1Password, Bitwarden)

#### Level 2: Password Policy Enforcement
```python
Requirements:
- Minimum length: 12 characters
- Character diversity: 3+ types (upper, lower, digit, symbol)
- Blacklist: Common passwords, company name
- No reuse: Check against previous 10 passwords
```

#### Level 3: Strong Hashing Algorithm
**Use Argon2id** with recommended parameters:
```python
from argon2 import PasswordHasher

ph = PasswordHasher(
    time_cost=3,        # 3 iterations
    memory_cost=65536,  # 64 MB RAM
    parallelism=4       # 4 threads
)

hash = ph.hash("user_password")
ph.verify(hash, "user_password")  # Raises exception if wrong
```

#### Level 4: Additional Defenses
1. **Rate limiting**: Max 5 login attempts per minute
2. **Account lockout**: Temporarily disable after 10 failed attempts
3. **2FA/MFA**: Require second factor (TOTP, SMS, hardware key)
4. **Monitoring**: Alert on suspicious login patterns
5. **Pepper**: Store secret pepper in HSM or separate config

### Time-Memory Trade-off
**Security goal**: Make cracking take longer than password lifetime

| Hash Time | Attacker Speed | 8-char [a-z] Crack Time |
|-----------|----------------|------------------------|
| 0.001s (MD5-crypt) | 1M/s | 3 hours |
| 0.01s (SHA256 10K) | 100K/s | 1 day |
| 0.1s (Argon2 low) | 10K/s | 3 weeks |
| 1.0s (Argon2 high) | 1K/s | 7 months |

**Recommendation**: Target 0.5-1 second per hash on production hardware

### Real-World Breach Examples
- **LinkedIn (2012)**: 6.5M SHA-1 hashes (no salt!) cracked in days
- **Adobe (2013)**: 150M passwords with weak encryption
- **Yahoo (2013)**: 3 billion accounts, bcrypt (better outcome)
- **RockYou (2009)**: 32M plaintext passwords (disaster!)

### Key Takeaways
1. ✅ Always use **Argon2** for new systems
2. ✅ Never store plaintext or reversible encryption
3. ✅ Unique salt per password, generated cryptographically
4. ✅ Long pepper (64+ chars), stored separately
5. ✅ Enforce strong password policies
6. ✅ Implement rate limiting and 2FA
7. ❌ Never use MD5, SHA-1, or plain SHA-256 for passwords
8. ❌ Don't roll your own crypto

## Practical Walkthrough

### Exercise Workflow

#### Step 1: Find MD5 Collision
```python
import hashlib

def find_md5_collision(prefix_length=6):
    seen = {}
    counter = 0
    
    while True:
        password = f"pass{counter}"
        hash_full = hashlib.md5(password.encode()).hexdigest()
        prefix = hash_full[:prefix_length]
        
        if prefix in seen:
            # Collision found!
            return seen[prefix], password, prefix
        
        seen[prefix] = password
        counter += 1
```

**Expected output**:
```
Collision found!
Password 1: pass862
Password 2: pass4708
Common prefix: b75278
```

#### Step 2: Crack MD5-crypt Hash
Given: `$1$saltyyyy$kR8qLKZw7LJ3ZNXhzN5Ql.`

```python
from passlib.hash import md5_crypt

def crack_md5_crypt(target_hash, dictionary):
    salt = target_hash.split('$')[2]  # 'saltyyyy'
    
    for password in dictionary:
        test_hash = md5_crypt.using(salt=salt).hash(password)
        
        if test_hash == target_hash:
            return password  # Found: "alibaba"
    
    return None
```

#### Step 3: Crack SHA256-crypt Hash
Given: `$5$rounds=5000$abcdefgh$xyz...`

```python
from passlib.hash import sha256_crypt

def crack_sha256_crypt(target_hash, dictionary):
    parts = target_hash.split('$')
    
    # Extract salt (handle rounds parameter)
    if 'rounds=' in parts[2]:
        salt = parts[3]
    else:
        salt = parts[2]
    
    for password in dictionary:
        test_hash = sha256_crypt.using(salt=salt).hash(password)
        
        if test_hash == target_hash:
            return password  # Found: "italy"
    
    return None
```

#### Step 4: Crack Argon2 Hash
Given: `$argon2id$v=19$m=16384,t=2,p=1$salt$hash`

```python
from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError

def crack_argon2(target_hash, dictionary):
    ph = PasswordHasher()
    
    for password in dictionary:
        try:
            ph.verify(target_hash, password)
            return password  # Found: "1951"
        except VerifyMismatchError:
            continue
    
    return None
```

#### Step 5: Crack Hash with Pepper
Given: Hash with unknown 1-char [a-z] pepper appended

```python
def crack_with_pepper(target_hash, dictionary, hash_function):
    for password in dictionary:
        for pepper in 'abcdefghijklmnopqrstuvwxyz':
            candidate = password + pepper
            
            if test_hash_match(candidate, target_hash, hash_function):
                return candidate  # Found: "maryannd" + "d"
    
    return None
```

### Expected Results

| Hash Type | Password | Time | Notes |
|-----------|----------|------|-------|
| MD5 collision | pass862, pass4708 | ~0.01s | Birthday paradox |
| MD5-crypt | alibaba | ~2s | Fast, weak scheme |
| SHA256-crypt | italy | ~10s | Moderate difficulty |
| Argon2 | 1951 | ~600s | Memory-hard, slow |
| Pepper (SHA256) | maryannd | ~300s | 26× multiplier |

### Performance Tips
1. **Precompile patterns**: Optimize dictionary loading
2. **Parallel processing**: Use multiprocessing for CPU-bound tasks
3. **GPU acceleration**: Use hashcat/john for serious cracking
4. **Progress tracking**: Show ETA for long-running attacks
5. **Early termination**: Stop immediately when password found